# **지능형 IoT 응용** (2021년 2학기)
#### 한국기술교육대학교 컴퓨터공학부 스마트 IoT 트랙


---

# 13. 에너지 사용 최적화

---

### Acknowledgement


이 자료는 다음 서적의 내용을 바탕으로 작성되었음
- 초소형 머신러닝 TinyML, 피트 워든, 대니얼 시투나야케 지음, 맹윤호, 임지순 옮김, 한빛미디어
 - 16장

---

- 임베디디 장치의 데스크톱이나 모바일 기기 대비 장점
 - 에너지 소모량이 작음

- 서버 CPU
 - 수십~수백 와트(W) 전력 소비
 - 냉각 시스템과 주 전원 공급 장치 필요

- 스마트폰
 - 수 와트(W) 소비
 - 매일 충전하다시피 해야 함

- 마이크로컨트롤러
 - 밀리와트(mW) 미만 
 - 휴대폰 CPU의 천 배 이하 전력으로 작동할 수 있음
 - 코인 배터리나 태양광 모듈을 써도 몇 주 혹은 몇 개월, 길게는 몇 년 동안도 작동할 수 있음

- TinyML 기반의 지능형 IoT 제품을 개발할 때 배터리 수명은 까다로운 문제임
 - 사람이 직접 배터리를 교체하거나 충전시킬 수 없는 경우가 많음
 - 장치의 실질적인 유효 수명(작동 지속 시간)은 전력 소모와 장치의 배터리 용량에 따라 결정됨
 - 배터리 용량은 일반적으로 제품의 실제 크기에 의해 제한됨
 - 태양광 충전 등 자가발전 기능을 사용할 수 있는 경우에도 얼마나 많은 전력을 공급할 수 있는지에 대한 한계가 존재
 - TinyML 제품의 수명은 시스템에서 소모하는 에너지에 의해 좌우됨

- 전력 사용량을 조사하고 개선하는 방법을 알아보자



## 13.1 직관 기르기

- 전력
 - 단위 시간당 쓰이는 에너지
 - 예를 들어 1초마다 1줄(J)의 에너지를 사용하는 CPU는 1W의 전력을 사용하는 것

- 전력 소모량은 배터리에 저장된 고정된 양의 에너지로 장치가 작동할 수 있는 시간에 반비례
 - 평균 1mW의 전력을 사용하는 시스템은 2mW를 사용하는 시스템보다 2배 오래 지속된다는 것을 쉽게 예측할 수 있음




### 13.1.1 일반적인 부품의 전력 소모

#### 전력 소모

- 스마트폰에 있는 대표적인 전자 부품의 전력 소모 예
 - "Smartphone Energy Consumption" (Cambridge University Press, 2014) 책의 계산을 따르면,
 - ARM Cortex-A9 CPU: 500 - 2000mW
 - 디스플레이: 400mW
 - 액티브 셀 라디오: 800mW
 - 블루투스: 100mW
- 이 외에 스마트폰 파워 소모에 대한 측정 및 분석, 모델링에 대한 많은 연구를 참고할 수 있음
 - https://www.usenix.org/legacy/event/usenix10/tech/full_papers/Carroll.pdf
 - https://dl.acm.org/doi/abs/10.1145/2796314.2745875
 

- 임베디드 부품의 소모 전력 예
 - 마이크 센서: 300uW
 - Bluetooth LE: 40mW
 - 320x320 픽셀 흑백 이미지 센서(예: Himax HM01B0): 1mW (30FPS 기준)
 - 앰비크 Cortex-M4F 마이크로컨트롤러: 1mW (48Hz 클럭 속도 기준)
 - 가속도계: 1mW

- 사용하는 부품 종류에 따라 전력 소모량이 크게 다르지만 이러한 자료가 있으면 최소한 서로 다른 기능 모듈의 대략적인 전력 소모 비율을 알 수 있음
 - 확실한 점 하나는 무선 통신 부품이 다른 임베디드 부품보다 훨씬 많은 전력을 사용한다는 점
 - 센서와 프로세서가 사용하는 전력량은 통신 부품이 사용하는 전력보다 훨씬 빠르게 줄어들고 있어서 향후에는 격차가 더 커질 것임


#### 에너지원

- 전원을 공급하기 위한 에너지원의 용량이나 발전량
 - CR2032 코인 배터리: 2500J 저장 (시스템에서 평균 1mW 전력을 사용하는 경우 약 1개월 동안 사용할 수 있음)
 - AA 배터리: 15000J 저장 (1mW를 쓰는 시스템의 경우 수명은 6개월)
 - 산업용 기계의 온도차를 이용한 발전: 1제곱cm당 대략 1-10mW
 - 실내 조명을 태양광 모듈로 받는 경우: 1제곱cm당 대략 10uW
 - 실외 조명을 태양광 모듈로 받는 경우: 1제곱cm당 대략 10mW



 

 


- 여기 제시된 숫자는 참고 사항일 뿐이지만 시스템 수명, 비용, 크기 등의 요구 사항의 조합에 따라 어떤 종류의 전력 시스템이 실용적인지 구상하는 데 도움이 될 것임
 - 이를 통해 초기 기획의 타당성을 확인할 수 있고 전력 소모에 대한 직관을 기를 수 있을 것임

### 13.1.2 하드웨어 선택

- 제품에 어떤 종류의 구성 요소를 사용할지 대략적으로 결정했다면 실제로 어떤 부품을 구입할지 선택해야 함

- 문서화가 잘 되어 있고 커뮤니티 정보가 많은 부품을 구하려면
 - 스파크펀(sparkfun): https://www.sparkfun.com/
 - 에이다프루트(adafruit): https://www.adafruit.com/
 - 아두이노(arduino): https://www.arduino.cc/
 - 튜토리얼, 드라이버, 추천 결합 부품이 잘 정리되어 있어서 프로토타입을 제작하기 좋음
 - 단점: 선택의 폭이 제한되고 통합된 시스템이 전체 전력 사용량에 맞게 최적화되지 않을 수 있음

- 고객지원이나 커뮤니티 정보, 문서화보다는 선택의 폭과 저렴한 가격을 선호한다면
 - 디지키(DigiKey): https://www.digikey.com/ 
 - 마우저 일렉트로닉스(Mouser Electronics): https://www.mouser.com/
 - 알리바바(Alibaba): https://www.alibaba.com/
 - 모든 제품에 대한 데이터시트를 제공. 여기에는 클록 신호 공급 방법부터 칩, 핀 크기 같은 기계적 데이터에 이르기까지 각 부품에 대한 자세한 정보가 포함돼 있음
 - 데이터시트에서 전력 사용량에 대한 정보를 찾아서 확인
 - mA 단위의 전류 소비량(current consumption)으로 나오기도 함 (1mW = 1mA x 1V)

- STMicroelectronics Cortex-M0 MCU 데이터시트 예
 - https://www.st.com/resource/en/datasheet/stm32f070c6.pdf
 - 전류 소비량에 동작 전압(3.6V)를 곱하면 전력 소모량

- 사용하고자 하는 부품의 데이터시트를 분석하고 모든 부품이 쓰는 전력 소모량의 합을 기준으로 전체 전력 사용량을 정리할 수 있음


## 13.2 실제 전력 소모 측정하기


- 부품을 다 모으면 전체 시스템을 구성할 수 있음
 - 실제로 제품을 시험해보고 요구 사항을 만족하는지 알아보려면 빨리 시스템을 완성하는 것이 좋음
 - 필요한 모든 부품이나 모든 소프트웨어를 준비하지 않았어도 빠르게 피드백을 얻는 것이 중요

- 완전한 시스템을 갖추면 실제 전력 사용량 테스트가 가능함
 - 데이터시트와 추정치는 계획에는 도움이 되지만 항상 실제와 일치하지는 않음
 - 통합 테스트를 수행하면 종종 예상보다 훨씬 높은 전력 소비를 보게 됨

#### 전력 소비를 측정하기 위한 다양한 방법

- 멀티미터를 사용하는 방법
 - 멀티미터: 전압, 전류, 저항 등 여러 가지 전기적 특성을 측정하는 장치

- 전용 파워 모니터를 사용하는 방법 
 - 예: Monsoon Power Monitor, https://www.msoon.com/high-voltage-power-monitor

 - 측정 사례 예시 이미지
<img src="https://www.researchgate.net/profile/Pawani-Porambage/publication/299845410/figure/fig4/AS:371410489888772@1465562569483/Power-measurement-of-wireless-sensor-node-using-Monsoon-power-monitor_Q640.jpg">

- 용량을 확인할 수 있는 배터리를 실제로 설치하고 장치 수명을 확인하는 방법
 - 부가적인 측정 장비가 필요하지 않고 시험 절차가 간단함
 - 전압이 너무 낮아질 때 발생하는 고장 등 실제 상황에서 발생할 수 있지만 간단한 모델링과 계산으로는 알기 힘든 문제도 포착할 수 있음
 - 배터리가 방전될 때까지 시간이 오래 걸릴 수 있음



## 13.3 모델의 전력 사용량 추정

- 모델이 특정 장치에서 사용할 전력량을 추정하는 가장 간단한 방법
 - 하나의 추론을 실행하기 위한 시간을 측정
 - 해당 기간 동안 시스템의 평균 전력 사용량 측정
 - 위 둘을 곱하여 에너지 사용량을 계산

- 모델을 실행하는 데 걸리는 시간의 추정
 - 모델에 필요한 산술연산의 수 (N)
 - 프로세서가 초당 수행할 수 있는 대략적인 연산의 수 (P)
 - N/P를 하면 대략적인 시간을 계산할 수 있음

- 전력 사용량의 추정
 - 데이터시트에서 제공하는 특정 주파수와 전압에서 소모 전력을 확인할 수 있음
 - 하지만 메모리나 주변 장치와 같은 전체 시스템의 공통 요소가 소모하는 전력까지는 포함하지 않을 수 있음
 - 전력 사용에 대한 추정치를 크게 잡아서 상한선으로 사용하면 적어도 접근 방식의 실현 가능성에 대해 파악할 수 있을 것

- 인체 감지 모델의 예
 - 모델이 6천만 건의 작업 실행
 - 48MHz에서 실행되는 ARM Cortex-M4 칩이 DSP 기능으로 한 사이클에 8비트 곱셈/덧셈을 2회씩 수행한다고 하정하면 최대 대기 시간은 48,000,000 / 60,000,000 = 800ms
 - 칩이 2mW를 소모한다면 추론 1회당 1.6mJ을 사용


